In [1]:
# https://python-binance.readthedocs.io/en/latest/overview.html
# https://www.binance.com/en/support/faq/c-6
# https://www.binance.com/en/my/settings/api-management

In [2]:
import requests
import numpy as np
import pandas as pd

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from futures_sign import send_signed_request, send_public_request
from cred import KEY, SECRET

In [3]:
symbol='ETHUSDT'
client = Client(KEY,SECRET)


In [4]:
# Get last 500 kandels 5 minutes for Symbol

def get_futures_klines(symbol,limit=500):
    x = requests.get('https://www.binance.com/fapi/v1/klines?symbol='+symbol+'&limit='+str(limit)+'&interval=5m')
    df=pd.DataFrame(x.json())
    df.columns=['open_time','open','high','low','close','volume','close_time','d1','d2','d3','d4','d5']
    df=df.drop(['d1','d2','d3','d4','d5'],axis=1)
    df['open']=df['open'].astype(float)
    df['high']=df['high'].astype(float)
    df['low']=df['low'].astype(float)
    df['close']=df['close'].astype(float)
    df['volume']=df['volume'].astype(float)
    print(df)
    return(df)

In [5]:
get_futures_klines(symbol)

         open_time     open     high      low    close     volume  \
0    1673461500000  1339.02  1340.50  1337.53  1339.70  19199.310   
1    1673461800000  1339.70  1342.90  1338.23  1338.34  32425.445   
2    1673462100000  1338.34  1339.07  1335.40  1336.38  32946.831   
3    1673462400000  1336.38  1337.45  1335.33  1337.20  19645.863   
4    1673462700000  1337.21  1338.28  1336.17  1336.37  14292.422   
..             ...      ...      ...      ...      ...        ...   
495  1673610000000  1408.86  1411.30  1407.84  1411.00  11749.247   
496  1673610300000  1411.00  1411.00  1407.04  1409.21  18040.890   
497  1673610600000  1409.21  1410.36  1408.82  1410.33   7659.908   
498  1673610900000  1410.33  1410.67  1408.49  1409.33   5670.749   
499  1673611200000  1409.33  1409.48  1407.12  1408.08  13484.575   

        close_time  
0    1673461799999  
1    1673462099999  
2    1673462399999  
3    1673462699999  
4    1673462999999  
..             ...  
495  1673610299999  
496

,open_time,open,high,low,close,volume,close_time
0,1673461500000,1339.02,1340.50,1337.53,1339.70,19199.310,1673461799999
1,1673461800000,1339.70,1342.90,1338.23,1338.34,32425.445,1673462099999
2,1673462100000,1338.34,1339.07,1335.40,1336.38,32946.831,1673462399999
3,1673462400000,1336.38,1337.45,1335.33,1337.20,19645.863,1673462699999
4,1673462700000,1337.21,1338.28,1336.17,1336.37,14292.422,1673462999999
...,...,...,...,...,...,...,...
495,1673610000000,1408.86,1411.30,1407.84,1411.00,11749.247,1673610299999
496,1673610300000,1411.00,1411.00,1407.04,1409.21,18040.890,1673610599999
497,1673610600000,1409.21,1410.36,1408.82,1410.33,7659.908,1673610899999
498,1673610900000,1410.33,1410.67,1408.49,1409.33,5670.749,1673611199999


In [6]:
# Open position for Sybol with 

def open_position(symbol,s_l,quantity_l):
    sprice=get_symbol_price(symbol)

    if(s_l=='long'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price        

                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
       
    if(s_l=='short'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
                }
           ]
        }
        responce = send_signed_request('POST', '/fapi/v1/batchOrders', params)
        

In [7]:
# Close position for symbol with quantity

def close_position(symbol,s_l,quantity_l):
    sprice=get_symbol_price(symbol)

    if(s_l=='long'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price
                }
        responce = send_signed_request('POST', '/fapi/v1/order', params)
    if(s_l=='short'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
                
                    "symbol":symbol,
                    "side": "BUY",
                    "type": "LIMIT",
                    "quantity": str(quantity_l),
                    "timeInForce":"GTC",
                    "price": close_price        
                }
        responce = send_signed_request('POST', '/fapi/v1/order', params)



In [10]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    a = positions[positions['symbol']==symbol]['positionAmt'].astype(float).tolist()[0]
    leverage = int(positions[positions['symbol']==symbol]['leverage'])
    entryprice = positions[positions['symbol']==symbol]['entryPrice']
    profit = float(status['totalUnrealizedProfit'])
    balance = round(float(status['totalWalletBalance']),2)
    if a>0:
        pos = "long"
    elif a<0:
        pos = "short"
    else: 
        pos = ""
    return([pos,a,profit,leverage,balance,round(float(entryprice),3),0])


In [11]:
get_opened_positions(symbol)

['', 0.0, 0.0, 5, 0.0, 0.0, 0]

In [12]:
# Close all orders 

def check_and_close_orders(symbol):
    global isStop 
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [13]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df[ df['symbol']==symbol]['price'])

In [12]:
open_position(symbol,'long',0.01)

In [ ]:
close_position(symbol,'long',0.02)

In [ ]:
get_opened_positions(symbol)